In [1]:
# Tentativo di analisi completa
# -----------------------------
# Import library

import re
import ast
import json
import nltk
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD


In [2]:
# Data loading
path = 'C:/Users/Davide/' # Sostituire il percorso quando si cambia location
data = []
with open(path + 'Corriere.txt') as f: # Sostituire il percorso quando si cambia location
    for line in f:
        data.append(json.loads(line))

        
collection_testi = []
for i in range(0, len(data)):  
    collection_testi.append(str(data[i]['title']) + str(data[i]['text']) + str(data[i]['tags']) + str(data[i]['resume']))
    

In [3]:
for i in range(0, len(collection_testi)):
    f= open("Corr%s.txt" %str(i) , "w+") #Il nome potrebbe cambiare
    f.write(collection_testi[i])
    f.close() 

In [4]:
# Routine for cleaning documents
def clean(path):
    with  open(path, 'r') as to_clean:
        one_line = ''
        for string in to_clean.readlines():
            one_line += string.lower()
    to_clean.close()
    
    #Special espressions of html format
    to_replace = ['\\n','\\t','\\r', '\\', '&nbsp'] 
    
    for item in to_replace:
        one_line = one_line.replace(item,' ') 

    cleaned = ' '.join(word for word in one_line.split() if len(word)>1)
    
    # All other special characters
    definitive = re.sub('[^a-zA-Zàéòùè]', ' ', cleaned)
    
    # Just the stopwords remain
    word_tokens = word_tokenize(definitive)
 
    filtered_text = [w for w in word_tokens if not w in stop_words]
    cleaned = ' '.join(word for word in filtered_text if len(word)>1)
    
    with open(path, 'w') as to_clean:
        to_clean.write(cleaned) 
        
    to_clean.close()
    
    return 

In [7]:
nltk.download("stopwords") 
stemmer = SnowballStemmer("italian") 
nltk.download('punkt')
stop_words = set(stopwords.words('italian'))
print('ok')
for i in range(0, len(collection_testi)):
    file_path = path + 'Corr' + str(i)+ '.txt' # Adesso arriva il bello...
    

    cleaned_text = clean(file_path)

    full_cleaned_text = str("")

    with open(file_path, 'r') as f:
        for line in f.readlines():
            full_cleaned_text = full_cleaned_text + " " + str(line)

    tokens = nltk.word_tokenize(full_cleaned_text)

    stemmed_text = ""

    for token in tokens:
        stemmed_text = stemmed_text + " " + stemmer.stem(token)
    
    with open(file_path, 'w') as f:
        f.write(stemmed_text)
    
#    print(stemmed_text)
    

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Davide\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Davide\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
ok
 matte figl fin acid riin dov mor anni fa rom mort tropp tard dov mor cinquant anni fa mormor incorregg dialett sicil santin matte pent maf confess strag capac cos dop prim dichiar magistr rap figl tredicenn giusepp uccis dop due anni segreg ex mafios colpevol tant omicid volt vittim maf adess dic content mort tot riin cos final chiud capitol cert segu sbagl fid famigl intorn madon ganc brusc forz appogg lasc tropp spaz rovin invec anni sessant vol togl mezz fatt giusepp ruffin corleon ved buon occhi nè nè caloger bagarell ammazz strag vial laz riin paur ruffin evit incontr poi ruffin mort lett riin ostacol fatt arrest luc ligg rimast sol comand ignor capr molt furb organizz

 lettur fa fest anni compleann sogn tradizion social giorn citt mil ritrov intorn libr bookcity fa ret coinvolg fin domen librer muse luog arte pass teatr fondazion quartier assoc ospit sal altri festival temp libr present altroier sal buzz festival metropol percors perifer mentr alcun incontr event ved collabor altre storic kermess milanesian dialog uom pistoi nuov numer de lettur edicol fin sab novembr numer special pagin dop scors settiman sol accompagn giorn manifest milanes propon altri nuov spunt riflession moltissim autor saggist pensator cominc straordinar presenz import fotoreporter guerr mond americ james nachtwey firm copertin numer fotograf finor ined dedic mur berlin maestr dedic mostr mil palazz real dicembr apertur numer prem streg francesc piccol propon riflession smisuratezz sogn sogn grand anzi grandissim scrittor offre altern interess addomestic fantas selvagg aspir altro autor interv enric rotell parl luog quas selvagg sogn bens campagn stermin po isol wisconsin rac

 torin insult razzist enne bus ferm uom enne disoccup torin uom scors settiman insult epitet razzist giul studentess color promess basket torines aggression verbal avven venerd scors intorn mattin bord bus numer denunc quindicenn uom insult inutil vai scuol tant fin strad tornaten paes stat carabinier rivol grugliasc graz collabor gtt grupp trasport torines militar acquis immagin telecam sorvegl mezz pubblic inform relat obliter passagg registr stess mezz giorn ora fatt riusc cos rintracc uom aggressor stat riconosc assol certezz vittim adess dovr rispond violenz mot razzial già conosc forz ordin pass stat segnal serviz territorial alcun episod caratterizz comport antisocial dov abus etil sostanz stupefacent sol alcun occasion già sottopost tratt sanitar obbligator luned scors promess trov colpevol atto volg violent pat possibil preg scus magistr invit sospett fars avant procur aggiunt patriz cap sostit eugen ghi giorn ier enne stat lung ascolt carabinier qual forn version fatt ricord 

In [ ]:
import findspark
findspark.init("/usr/local/spark") # Potrebbe cambiare
import pyspark
from pyspark import SparkContext
sc =SparkContext()
from pyspark.sql.session import SparkSession
spark = SparkSession(sc)

In [ ]:
from pyspark.sql import SQLContext, Row
from pyspark.ml.feature import CountVectorizer
from pyspark.mllib.clustering import LDA, LDAModel
from pyspark.mllib.linalg import Vector, Vectors
from pyspark.ml.feature import StopWordsRemover

lista_topic = []
lista_pesi = []

for i in range(0, len(collection_testi)):
    file_path = path + 'Corr' + str(i)+ '.txt' # Modificare se è necessario

    data = sc.textFile(file_path).zipWithIndex().map( lambda cleanwords_idd: Row(idd = cleanwords_idd[1], cleanwords = cleanwords_idd[0].split(" ")))
    docDF = spark.createDataFrame(data)


    Vector = CountVectorizer(inputCol="cleanwords", outputCol="vectors")
    model = Vector.fit(docDF)
    result = model.transform(docDF)

    corpus = result.select("idd", "vectors").rdd.map(lambda x_y: [x_y[0],Vectors.fromML(x_y[1])]).cache()

    # Cluster the documents into three topics using LDA
    ldaModel = LDA.train(corpus, k=3,maxIterations=100,optimizer='online')
    topics = ldaModel.topicsMatrix()
    vocabArray = model.vocabulary

    weight = ldaModel.describeTopics(maxTermsPerTopic = 10)

    wordNumbers = 10  # number of words per topic
    topicIndices = sc.parallelize(ldaModel.describeTopics(maxTermsPerTopic = wordNumbers))

    def topic_render(topic):  # specify vector id of words to actual words
        terms = topic[0]
        result = []
        for i in range(wordNumbers):
            term = vocabArray[terms[i]]
            result.append(term)
        return result

    topics_final = topicIndices.map(lambda topic: topic_render(topic)).collect()

    #print(vocabArray(int(weight[0][0][0])))
    i = 0 
    terms = ''
    for topic in range(len(topics_final)):
        #print ("Topic" + str(topic) + ":")
        j = 0
        for term in topics_final[topic]:
            terms = terms + ' ' + term
            lista_pesi.append(weight[i][1][j])
            j += 1
        i += 1
        lista_topic.append(terms)
        

In [ ]:
# Comprendere a che cosa può servire la lista pesi (magari accanto alla lista topic ce ne mettiamo un'altra)
# Calibrare il numero di parametri (numero parole e topic clusters)

In [ ]:
vectorizer = TfidfVectorizer(use_idf=True, ngram_range=(1, 3))
X = vectorizer.fit_transform(lista_topic) # E' una lista 

lsa = TruncatedSVD(n_components = 2, n_iter = 1000) # Qual è il numero più corretto di componenti???
lsa.fit(X)

In [ ]:
terms = vectorizer.get_feature_names()
out = []
for i, comp in enumerate(lsa.components_):
    termsInComp = zip(terms,comp)
    sortedTerms = sorted(termsInComp, key=lambda x: x[1], reverse=True ) [:10]
    string = ''
    
    for term in sortedTerms:
        string = string + ' ' + term[0]
    out.append(string)

In [ ]:
# Salvataggio
with open("Output News.txt", "w") as output:
    output.write("\n".join(map(lambda x: str(x), out)))